## Resolución Team Miss Marple

#### SQL Murder

In [22]:
# Usaremos las bibliotecas sqlite3 y pandas para trabajar con la base de datos
import pandas as pd
import sqlite3

In [24]:
# Estableciendo conexión con la base de datos
conn = sqlite3.connect('./data/sql-murder-mystery.db')

# Configuramos pandas para que muestre todas las columnas
pd.set_option('display.max_colwidth', None)

In [25]:
# Transformaremos todas las tablas SQL en DataFrame homónimos de pandas, 
# para facilitar la visualización de los datos y posibles análisis.

query = "SELECT name FROM sqlite_master WHERE type='table';"
tablas = pd.read_sql(query, conn)

# Crear y ejecutar la query, DataFrame para obtener los nombres de las tablas
for nombre_tabla in tablas['name']:
    query = f"SELECT * FROM {nombre_tabla}"
    globals()[nombre_tabla] = pd.read_sql(query, conn)  

In [26]:
# Mirando qué aspecto tiene el DataFrame 'crime_scene_report'.
crime_scene_report.sample(5)

,date,type,description,city
1076,20180228,assault,\n,Frederick
869,20171204,assault,and broke to pieces against one of the trees behind him.\n,Killeen
486,20170629,smuggling,look through into the garden with one eye; but to get through was more\n,Atlantic City
494,20170704,theft,\n,Davidson County
426,20170610,bribery,the best thing to get us dry would be a Caucus-race.’\n,Knoxville


In [27]:
crime_scene_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1228 non-null   int64 
 1   type         1228 non-null   object
 2   description  1228 non-null   object
 3   city         1228 non-null   object
dtypes: int64(1), object(3)
memory usage: 38.5+ KB


#### A partir de aquí resolveremos la investigación en dos enfoques diferentes:

***(1) directamente con consultas SQL;***

***(2) utilizando los DataFrames de pandas.***

In [28]:
# (1) Query SQL
# Recuperando el informe 

query = '''
SELECT description 
FROM crime_scene_report
WHERE date = '20180115' AND type = 'murder' AND city = 'SQL City';
'''
pd.read_sql_query(query, conn)

,description
0,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


In [29]:
# (2) DataFrame en Pandas

mascara = (crime_scene_report["date"] == 20180115) & \
          (crime_scene_report["city"] == "SQL City") & \
          (crime_scene_report["type"] == "murder")

condicion = crime_scene_report[mascara]
condicion

,date,type,description,city
1227,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [30]:
# (1) Query SQL

query = '''
SELECT id, name
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'
'''
pd.read_sql_query(query, conn)

,id,name
0,16371,Annabel Miller


In [31]:
# (2) DataFrame Pandas

mascara = ((person["name"].str.startswith("Annabel")) & \
           (person["address_street_name"] == "Franklin Ave"))

condicion = person[mascara]
condicion

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [32]:
# (1) Query SQL

query = '''
SELECT id, name
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC 
LIMIT 1
'''
pd.read_sql_query(query, conn)

,id,name
0,14887,Morty Schapiro


In [11]:
# (2) DataFrame Pandas

mascara = (person["address_street_name"] == "Northwestern Dr")

condicion = person[mascara]
sorted_adress = condicion.sort_values(by="address_number", ascending=False)
sorted_adress.head(1)

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


#### Ahora que encontramos a los dos testigos del crimen, Annabel Miller y Morty Schapiro, vamos a buscar sus testimonios.

In [12]:
# (1) Query SQL

query = '''
SELECT transcript
FROM interview
WHERE person_id = '16371'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [13]:
# (2) DataFrame Pandas

mascara = (interview["person_id"] == 16371)

condicion = interview[mascara]
condicion

,person_id,transcript
4989,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [14]:
# (1) Query SQL

query = '''
SELECT transcript
FROM interview
WHERE person_id = '14887'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [15]:
# (2) DataFrame Pandas

mascara = (interview["person_id"] == 14887)

condicion = interview[mascara]
condicion

,person_id,transcript
4988,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


#### Ahora buscaremos a un hombre con membresía Gold del gimnasio, con número de matrícula que contenga 48Z, con matrícula de coche que contenga H42W, y con fecha de acceso al gimnasio el 1 de enero de 2018.

In [17]:
# (1) Query SQL

query = '''
SELECT membership_id
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%' AND check_in_date = '20180109'
'''
pd.read_sql_query(query, conn)

,membership_id
0,48Z7A
1,48Z55


In [18]:
# (2) DataFrame Pandas

mascara = ((get_fit_now_check_in["membership_id"].str.startswith("48Z")) & \
           (get_fit_now_check_in["check_in_date"] == 20180109))

condicion = get_fit_now_check_in[mascara]
condicion

,membership_id,check_in_date,check_in_time,check_out_time
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [19]:
# (1) Query SQL

query = '''
SELECT *
FROM get_fit_now_member
WHERE id = '48Z7A' OR id='48Z55'
'''
pd.read_sql_query(query, conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


In [20]:
# (2) DataFrame Pandas

mascara = ((get_fit_now_member["id"] == "48Z7A") | \
           (get_fit_now_member["id"] == "48Z55"))

condicion = get_fit_now_member[mascara]
condicion

,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [21]:
# (1) Query SQL

query = '''
SELECT p.name, d.plate_number
FROM person AS p
JOIN drivers_license AS d ON p.license_id = d.id
WHERE p.name = 'Joe Germuska' OR p.name = 'Jeremy Bowers'H42W
'''
pd.read_sql_query(query, conn)

DatabaseError: Execution failed on sql '
SELECT p.name, d.plate_number
FROM person AS p
JOIN drivers_license AS d ON p.license_id = d.id
WHERE p.name = 'Joe Germuska' OR p.name = 'Jeremy Bowers'H42W
': near "H42W": syntax error

In [ ]:
# (2) DataFrame Pandas

merged_df = pd.merge(person, drivers_license, left_on='license_id', right_on='id', how='inner')

df_condicion = merged_df[merged_df['name'].isin(['Joe Germuska', 'Jeremy Bowers'])]

result = df_condicion[['name', 'plate_number']]
result

,name,plate_number
6323,Jeremy Bowers,0H42W2


#### El asesino es Jeremy Bowers. Aahora buscaremos a la persona que encargó el crimen.

In [ ]:
# (1) Query SQL

query = '''
SELECT transcript
FROM interview
WHERE person_id = '67318'
'''
pd.read_sql_query(query, conn)

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [ ]:
# (2) DataFrame Pandas

mascara = interview["person_id"] == 67318

condicion = interview[mascara]
condicion

,person_id,transcript
4990,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [100]:
# (1) Query SQL

query = '''
SELECT p.name, d.height, d.hair_color, d.car_make, d.car_model, d.gender
FROM drivers_license AS d
JOIN person AS p ON d.id = p.license_id
WHERE d.height BETWEEN 65 AND 67
AND d.hair_color = 'red'
AND d.gender = 'female'
AND d.car_make = 'Tesla'
AND d.car_model = 'Model S'
AND p.id IN (
	SELECT f.person_id
	FROM facebook_event_checkin AS f
	WHERE f.event_name = 'SQL Symphony Concert'
);
'''
pd.read_sql_query(query, conn)

,name,height,hair_color,car_make,car_model,gender
0,Miranda Priestly,66,red,Tesla,Model S,female


In [ ]:
# (2) DataFrame Pandas

merged = pd.merge(drivers_license, person, left_on='id', right_on='license_id', how="inner")
merged.columns

Index(['id_x', 'age', 'height', 'eye_color', 'hair_color', 'gender',
       'plate_number', 'car_make', 'car_model', 'id_y', 'name', 'license_id',
       'address_number', 'address_street_name', 'ssn'],
      dtype='object')

In [114]:
condicion = merged[
    (merged['height'].between(65, 67)) &  
    (merged['hair_color'] == 'red') &  
    (merged['gender'] == 'female') &  
    (merged['car_make'] == 'tesla') &  
    (merged['car_model'] == 'model s') 
]
event_person_ids = facebook_event_checkin[facebook_event_checkin['event_name'] == 'SQL Symphony Concert']['person_id']
condicion = condicion[condicion['id_y'].isin(event_person_ids)]
condicion["name"]

1104    miranda priestly
Name: name, dtype: object

#### Miranda Priestly es la persona quién encargó el crimen

#### En resumen, en la siguiente tabla se muestra a los testigos, al asesino, y quién encargó el crimen

| Rol        | Nombre              |
|--------------|-------------------|
| Testigos  | Annabel Miller, Morty Schapiro |
| Asesino   | Jeremy Bowers     |
| Mandante    | Miranda Priestly  |